# Main Feedback Generation Notebook
This notebook handles preprocessing, model interaction, and feedback generation.

**--Set up: Github, Paths, Imports**

In [1]:
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
!git branch -a

/content
fatal: destination path 'project' already exists and is not an empty directory.
/content/project
  main
* scraping/rubric_table
  remotes/origin/16-steps-for-phase-1
  remotes/origin/HEAD -> origin/main
  remotes/origin/feat/create-gradio-ui
  remotes/origin/feat/preprocess-the-data-into-a-normalized-format
  remotes/origin/fix/imports-and-file-paths
  remotes/origin/main
  remotes/origin/prompting/Deepseek
  remotes/origin/prompting/Gemini
  remotes/origin/refactor/docs-and-APIs
  remotes/origin/refactor/py-and-output-documents
  remotes/origin/scraping/rubric_table
  remotes/origin/wip/prompting


In [2]:
# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout scraping/rubric_table

M	requirements.txt
M	tropos/preprocess_docx/rubric.py
Already on 'scraping/rubric_table'
Your branch is up to date with 'origin/scraping/rubric_table'.


**--Install required libraries and set paths**

In [3]:
%pip install -r requirements.txt
!pip install python-docx

# import data path for loading files
import os

# Base data directory
data_base = '/content/project/data'

# Paths to specific subfolders
raw_data_path = os.path.join(data_base, 'raw')
processed_data_path = os.path.join(data_base, 'processed')

**--Import modules (youre working on)**
<br>*each of our classes will be what will merge to this notebook (im p sure)*
<br>only loads what you explicitly request
<br>(this helps keep memory low and import fast)
<br> *the following is an example with my Rubric module*

## Rubric scraper

In [4]:
# Add your src folder to the Python path
import sys
sys.path.append('/content/project/')

from tropos.preprocess_docx.rubric import Rubric, parse_rubric, extract_highlighted_phrases

from docx import Document

# loop through tables and cells
doc_path = "/content/project/data/raw/Student 1/Student 1 Part 1.docx"
rubric = parse_rubric(doc_path)
highlights = extract_highlighted_phrases(doc_path)


In [5]:
for h in highlights:
    print(f'"{h["text"]}" → {h["highlight"]}')

for row in rubric.get_criteria():
    print("Portion:", row["portion"])
    print("Criteria:", row["criteria"]["text"])

    row_highs = row["criteria"].get("highlights", [])
    if row_highs:
        print("Highlights:")
        for h in row_highs:
            print(f"  - \"{h['text']}\" (color: {h['highlight']})")
    else:
        print("Highlights: None")

    print("Feedback:", row["feedback"])
    print("-" * 50)

"Uses credible, neutral sources" → green
"Effectively and sufficiently explains essential information" → yellow
"Written objectively in own words" → yellow
"Well organized and focused paragraphs with transitions" → red
"Minimal error" → yellow
Portion: Introduction
Criteria: Engaging
Shows importance of question
Indicates major parts and content
Highlights:
  - "Uses credible, neutral sources" (color: green)
  - "Effectively and sufficiently explains essential information" (color: yellow)
  - "Written objectively in own words" (color: yellow)
  - "Well organized and focused paragraphs with transitions" (color: red)
  - "Minimal error" (color: yellow)
Feedback: 
--------------------------------------------------
Portion: Background
Criteria: Uses credible, neutral sources
Effectively and sufficiently explains essential information
Written objectively in own words
Well organized and focused paragraphs with transitions
Minimal error
Highlights:
  - "Uses credible, neutral sources" (color: